# Conversion Tools

https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4

<img src="blog-2019-12-converting-whole-slide-images.jpg" style="height:300px" />



## Basics

The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.

In [1]:
%%time
!bioformats2raw --help

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [--debug] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--compression-parameter=<compressionParameter>]
                    [--dimension-order=<dimensionOrder>]
                    [--file_type=<fileType>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>] [--pyramid-name=<pyramidName>]
                    [-r=<pyramidResolutions>]
                    [--scale-format-string=<scaleFormatString>]
                    [-w=<tileWidth>] <inputPath> <outputPath>
      <inputPath>    file to convert
      <outputPath>   path to the output pyramid directory
      --additional-scale-format-string-args=<additionalScaleFormatStringArgsCsv>
                    

In [2]:
%%time
!raw2ometiff --help

Missing required parameters: <inputDirectory>, <outputFilePath>
Usage: <main class> [--debug] [--legacy] [--rgb] [--compression=<compression>]
                    [--max_workers=<maxWorkers>] <inputDirectory>
                    <outputFilePath>
      <inputDirectory>   Directory containing pixel data to convert
      <outputFilePath>   Relative path to the output OME-TIFF file
      --compression=<compression>
                         Compression type for output OME-TIFF file
                           (Uncompressed, LZW, JPEG-2000, JPEG-2000 Lossy,
                           JPEG, zlib; default: LZW)
      --debug            Turn on debug logging
      --legacy           Write a Bio-Formats 5.9.x pyramid instead of OME-TIFF
      --max_workers=<maxWorkers>
                         Maximum number of workers (default: 16)
      --rgb              Attempt to write channels as RGB; channel count must
                           be 3
CPU times: user 8.78 ms, sys: 6.95 ms, total: 15.7 ms
Wa

## Simple Invocation

Here we will use Bio-Formats ability to generate test data in order to quickly test within the notebook.

See https://docs.openmicroscopy.org/bio-formats/latest/developers/generating-test-images.html for more information.

**First we generate the raw intermediate format:**

In [3]:
%%time
!bioformats2raw a.fake /tmp/output-1

2020-05-28 09:03:43,097 [main] INFO  loci.formats.ImageReader - FakeReader initializing a.fake
2020-05-28 09:03:43,214 [main] INFO  c.g.bioformats2raw.Converter - Using 2 pyramid resolutions
2020-05-28 09:03:43,214 [main] INFO  c.g.bioformats2raw.Converter - Preparing to write pyramid sizeX 512 (tileWidth: 1024) sizeY 512 (tileWidth: 1024) imageCount 1
2020-05-28 09:03:43,447 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileWidth to 512
2020-05-28 09:03:43,447 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileHeight to 512
2020-05-28 09:03:43,459 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 0, 0, 0] to /0/0
2020-05-28 09:03:43,469 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 1/1
2020-05-28 09:03:43,469 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1590649423459] time[9] tag[getTile]
2020-05-28 09:03:43,472 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully w

**Then we convert that output into an OME-TIFF:**

In [4]:
%%time
!raw2ometiff /tmp/output-1 /tmp/output-1.ome.tiff

2020-05-28 09:03:45,781 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Creating tiled pyramid file /tmp/output-1.ome.tiff
2020-05-28 09:03:45,831 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Number of resolution levels: 2
2020-05-28 09:03:45,838 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 0
2020-05-28 09:03:45,839 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 1
2020-05-28 09:03:45,841 [main] INFO  org.perf4j.TimingLogger - start[1590649425536] time[304] tag[initialize]
2020-05-28 09:03:45,892 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Converting resolution #0
2020-05-28 09:03:45,894 [main] INFO  org.perf4j.TimingLogger - start[1590649425892] time[1] tag[getInputTileBytes]
2020-05-28 09:03:45,895 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Converting resolution #1
2020-05-28 09:03:45,898 [main] INFO  org.perf4j.TimingLogger - start[1590649425895] time[2] tag[getInputTileBytes]
2020-05-28 09:03:45,902 [poo

**The same operation on larger generated data, still proceeds timely:**

In [5]:
%%time
!bioformats2raw 'big&sizeX=10000&sizeY=10000.fake' /tmp/output-2

2020-05-28 09:03:47,883 [main] INFO  loci.formats.ImageReader - FakeReader initializing big&sizeX=10000&sizeY=10000.fake
2020-05-28 09:03:48,004 [main] INFO  c.g.bioformats2raw.Converter - Using 7 pyramid resolutions
2020-05-28 09:03:48,004 [main] INFO  c.g.bioformats2raw.Converter - Preparing to write pyramid sizeX 10000 (tileWidth: 1024) sizeY 10000 (tileWidth: 1024) imageCount 1
2020-05-28 09:03:48,235 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [1, 0, 0, 0, 0] to /0/0
2020-05-28 09:03:48,235 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [4, 1, 0, 0, 0] to /0/0
2020-05-28 09:03:48,260 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - tile read complete 2/100
2020-05-28 09:03:48,260 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 2/100
2020-05-28 09:03:48,260 [pool-1-thread-15] INFO  org.perf4j.TimingLogger - start[1590649428235] time[24] tag[getTile]
2020-05-28 09:03:48,260 [

2020-05-28 09:03:48,393 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - successfully wrote at [1, 3, 0, 0, 0] to /0/0
2020-05-28 09:03:48,393 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=1024 yy=3072 width=1024 height=1024
2020-05-28 09:03:48,395 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [2, 3, 0, 0, 0] to /0/0
2020-05-28 09:03:48,400 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - tile read complete 19/100
2020-05-28 09:03:48,401 [pool-1-thread-8] INFO  org.perf4j.TimingLogger - start[1590649428395] time[5] tag[getTile]
2020-05-28 09:03:48,402 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - successfully wrote at [2, 3, 0, 0, 0] to /0/0
2020-05-28 09:03:48,402 [pool-1-thread-8] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=2048 yy=3072 width=1024 height=1024
2020-05-28 09:03:48,403 [pool-1-thread-8] INFO  c.g.bioformats

2020-05-28 09:03:48,497 [pool-1-thread-10] INFO  c.g.bioformats2raw.Converter - tile read complete 39/100
2020-05-28 09:03:48,497 [pool-1-thread-10] INFO  org.perf4j.TimingLogger - start[1590649428490] time[7] tag[getTile]
2020-05-28 09:03:48,498 [pool-1-thread-11] INFO  c.g.bioformats2raw.Converter - tile read complete 40/100
2020-05-28 09:03:48,498 [pool-1-thread-11] INFO  org.perf4j.TimingLogger - start[1590649428490] time[7] tag[getTile]
2020-05-28 09:03:48,500 [pool-1-thread-10] INFO  c.g.bioformats2raw.Converter - successfully wrote at [1, 5, 0, 0, 0] to /0/0
2020-05-28 09:03:48,500 [pool-1-thread-10] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=1024 yy=5120 width=1024 height=1024
2020-05-28 09:03:48,500 [pool-1-thread-11] INFO  c.g.bioformats2raw.Converter - successfully wrote at [2, 5, 0, 0, 0] to /0/0
2020-05-28 09:03:48,501 [pool-1-thread-11] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=

2020-05-28 09:03:48,596 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - successfully wrote at [2, 8, 0, 0, 0] to /0/0
2020-05-28 09:03:48,596 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=2048 yy=8192 width=1024 height=1024
2020-05-28 09:03:48,596 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [4, 8, 0, 0, 0] to /0/0
2020-05-28 09:03:48,597 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [5, 8, 0, 0, 0] to /0/0
2020-05-28 09:03:48,599 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - tile read complete 71/100
2020-05-28 09:03:48,599 [pool-1-thread-6] INFO  org.perf4j.TimingLogger - start[1590649428594] time[5] tag[getTile]
2020-05-28 09:03:48,600 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - successfully wrote at [3, 8, 0, 0, 0] to /0/0
2020-05-28 09:03:48,600 [pool-1-thread-6] INFO  c.g.bioformats2raw.Converter - Successfully proc

2020-05-28 09:03:48,648 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 89/100
2020-05-28 09:03:48,648 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1590649428643] time[5] tag[getTile]
2020-05-28 09:03:48,650 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [3, 0, 0, 0, 0] to /0/0
2020-05-28 09:03:48,650 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=0 xx=3072 yy=0 width=1024 height=1024
2020-05-28 09:03:48,663 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [6, 6, 0, 0, 0] to /0/0
2020-05-28 09:03:48,669 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 90/100
2020-05-28 09:03:48,669 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1590649428663] time[5] tag[getTile]
2020-05-28 09:03:48,671 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully wrote at [6, 6, 0, 0, 0] to /0/0
2020-05-28 0

2020-05-28 09:03:49,012 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1590649429002] time[9] tag[getTileDownsampled]
2020-05-28 09:03:49,012 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - tile read complete 4/25
2020-05-28 09:03:49,012 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1590649429002] time[10] tag[getTile]
2020-05-28 09:03:49,014 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - successfully wrote at [3, 3, 0, 0, 0] to /0/1
2020-05-28 09:03:49,014 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=1 plane=0 xx=3072 yy=3072 width=1024 height=1024
2020-05-28 09:03:49,015 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [4, 3, 0, 0, 0] to /0/1
2020-05-28 09:03:49,024 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1590649429015] time[8] tag[getTileDownsampled]
2020-05-28 09:03:49,024 [pool-1-thread-5] INFO  c.g.bioformats2raw.Converter - tile read complete 5/25

2020-05-28 09:03:49,258 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [4, 0, 0, 0, 0] to /0/1
2020-05-28 09:03:49,259 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 3, 0, 0, 0] to /0/1
2020-05-28 09:03:49,263 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [2, 1, 0, 0, 0] to /0/1
2020-05-28 09:03:49,269 [pool-1-thread-7] INFO  org.perf4j.TimingLogger - start[1590649429259] time[10] tag[getTileDownsampled]
2020-05-28 09:03:49,269 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - tile read complete 17/25
2020-05-28 09:03:49,269 [pool-1-thread-7] INFO  org.perf4j.TimingLogger - start[1590649429259] time[10] tag[getTile]
2020-05-28 09:03:49,271 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 3, 0, 0, 0] to /0/1
2020-05-28 09:03:49,271 [pool-1-thread-7] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=1 plane=0 xx=

2020-05-28 09:03:49,686 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [2, 0, 0, 0, 0] to /0/2
2020-05-28 09:03:49,686 [pool-1-thread-10] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [1, 0, 0, 0, 0] to /0/2
2020-05-28 09:03:49,690 [pool-1-thread-9] INFO  org.perf4j.TimingLogger - start[1590649429686] time[3] tag[getTileDownsampled]
2020-05-28 09:03:49,690 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - tile read complete 4/9
2020-05-28 09:03:49,690 [pool-1-thread-9] INFO  org.perf4j.TimingLogger - start[1590649429686] time[3] tag[getTile]
2020-05-28 09:03:49,691 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - successfully wrote at [2, 0, 0, 0, 0] to /0/2
2020-05-28 09:03:49,691 [pool-1-thread-9] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=2 plane=0 xx=2048 yy=0 width=452 height=1024
2020-05-28 09:03:49,789 [pool-1-thread-15] INFO  c.g.bioformats2raw.Converter - requesting tile to writ

CPU times: user 87.7 ms, sys: 32.5 ms, total: 120 ms
Wall time: 3.52 s


**But of course produces larger output:**

In [6]:
%%time
!raw2ometiff /tmp/output-2 /tmp/output-2.ome.tiff

2020-05-28 09:03:51,378 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Creating tiled pyramid file /tmp/output-2.ome.tiff
2020-05-28 09:03:51,430 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Number of resolution levels: 7
2020-05-28 09:03:51,443 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 0
2020-05-28 09:03:51,444 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 1
2020-05-28 09:03:51,445 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 2
2020-05-28 09:03:51,445 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 3
2020-05-28 09:03:51,445 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 4
2020-05-28 09:03:51,445 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 5
2020-05-28 09:03:51,445 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 6
2020-05-28 09:03:51,447 [main] I

2020-05-28 09:03:51,667 [pool-1-thread-14] INFO  org.perf4j.TimingLogger - start[1590649431643] time[23] tag[writeTile]
2020-05-28 09:03:51,667 [pool-1-thread-12] INFO  org.perf4j.TimingLogger - start[1590649431642] time[24] tag[writeTile]
2020-05-28 09:03:51,668 [pool-1-thread-16] INFO  org.perf4j.TimingLogger - start[1590649431642] time[25] tag[writeTile]
2020-05-28 09:03:51,668 [main] INFO  org.perf4j.TimingLogger - start[1590649431646] time[21] tag[getInputTileBytes]
2020-05-28 09:03:51,668 [pool-1-thread-13] INFO  org.perf4j.TimingLogger - start[1590649431643] time[25] tag[writeTile]
2020-05-28 09:03:51,668 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1590649431642] time[26] tag[writeTile]
2020-05-28 09:03:51,669 [pool-1-thread-7] INFO  org.perf4j.TimingLogger - start[1590649431643] time[25] tag[writeTile]
2020-05-28 09:03:51,669 [main] INFO  org.perf4j.TimingLogger - start[1590649431668] time[1] tag[getInputTileBytes]
2020-05-28 09:03:51,670 [pool-1-thread-1] INFO  org

2020-05-28 09:03:51,816 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1590649431804] time[12] tag[writeTile]
2020-05-28 09:03:51,818 [main] INFO  org.perf4j.TimingLogger - start[1590649431816] time[2] tag[getInputTileBytes]
2020-05-28 09:03:51,818 [pool-1-thread-13] INFO  org.perf4j.TimingLogger - start[1590649431805] time[13] tag[writeTile]
2020-05-28 09:03:51,818 [pool-1-thread-9] INFO  org.perf4j.TimingLogger - start[1590649431806] time[12] tag[writeTile]
2020-05-28 09:03:51,818 [pool-1-thread-10] INFO  org.perf4j.TimingLogger - start[1590649431805] time[12] tag[writeTile]
2020-05-28 09:03:51,818 [pool-1-thread-11] INFO  org.perf4j.TimingLogger - start[1590649431805] time[13] tag[writeTile]
2020-05-28 09:03:51,819 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1590649431807] time[12] tag[writeTile]
2020-05-28 09:03:51,819 [main] INFO  org.perf4j.TimingLogger - start[1590649431818] time[1] tag[getInputTileBytes]
2020-05-28 09:03:51,819 [main] INFO  c.g.p.PyramidFro

2020-05-28 09:03:51,918 [main] INFO  org.perf4j.TimingLogger - start[1590649431447] time[470] tag[convertToPyramid]
CPU times: user 36.5 ms, sys: 17.4 ms, total: 53.9 ms
Wall time: 1.47 s


In [7]:
!ls -ltrah /tmp/output*tiff

-rw-r--r--  1 jamoore  wheel    54K May 28 09:03 /tmp/output-1.ome.tiff
-rw-r--r--  1 jamoore  wheel   138M May 28 09:03 /tmp/output-2.ome.tiff


## Compression

Both commands additionally provide additional arguments like `--compression` which you can experiment with:

**Here we leave the OME-TIFF uncompressed to see how much larger it will be:**

In [8]:
%%time
!raw2ometiff /tmp/output-2 /tmp/output-2.ome.tiff --compression=Uncompressed

2020-05-28 09:03:56,536 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Creating tiled pyramid file /tmp/output-2.ome.tiff
2020-05-28 09:03:56,586 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Number of resolution levels: 7
2020-05-28 09:03:56,601 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 0
2020-05-28 09:03:56,602 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 1
2020-05-28 09:03:56,602 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 2
2020-05-28 09:03:56,602 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 3
2020-05-28 09:03:56,602 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 4
2020-05-28 09:03:56,602 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 5
2020-05-28 09:03:56,602 [main] INFO  c.g.p.PyramidFromDirectoryWriter - Adding metadata for resolution: 6
2020-05-28 09:03:56,604 [main] I

2020-05-28 09:03:56,759 [main] INFO  org.perf4j.TimingLogger - start[1590649436757] time[2] tag[getInputTileBytes]
2020-05-28 09:03:56,760 [pool-1-thread-10] INFO  org.perf4j.TimingLogger - start[1590649436760] time[0] tag[writeTile]
2020-05-28 09:03:56,761 [main] INFO  org.perf4j.TimingLogger - start[1590649436760] time[1] tag[getInputTileBytes]
2020-05-28 09:03:56,762 [pool-1-thread-11] INFO  org.perf4j.TimingLogger - start[1590649436762] time[0] tag[writeTile]
2020-05-28 09:03:56,763 [main] INFO  org.perf4j.TimingLogger - start[1590649436761] time[1] tag[getInputTileBytes]
2020-05-28 09:03:56,765 [pool-1-thread-12] INFO  org.perf4j.TimingLogger - start[1590649436763] time[1] tag[writeTile]
2020-05-28 09:03:56,765 [main] INFO  org.perf4j.TimingLogger - start[1590649436763] time[2] tag[getInputTileBytes]
2020-05-28 09:03:56,766 [pool-1-thread-13] INFO  org.perf4j.TimingLogger - start[1590649436765] time[0] tag[writeTile]
2020-05-28 09:03:56,767 [main] INFO  org.perf4j.TimingLogger - s

2020-05-28 09:03:56,809 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1590649436808] time[1] tag[writeTile]
2020-05-28 09:03:56,810 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1590649436809] time[0] tag[writeTile]
2020-05-28 09:03:56,810 [main] INFO  org.perf4j.TimingLogger - start[1590649436809] time[0] tag[getInputTileBytes]
2020-05-28 09:03:56,811 [pool-1-thread-5] INFO  org.perf4j.TimingLogger - start[1590649436810] time[0] tag[writeTile]
2020-05-28 09:03:56,811 [main] INFO  org.perf4j.TimingLogger - start[1590649436810] time[1] tag[getInputTileBytes]
2020-05-28 09:03:56,812 [pool-1-thread-6] INFO  org.perf4j.TimingLogger - start[1590649436811] time[0] tag[writeTile]
2020-05-28 09:03:56,812 [main] INFO  org.perf4j.TimingLogger - start[1590649436811] time[0] tag[getInputTileBytes]
2020-05-28 09:03:56,813 [pool-1-thread-7] INFO  org.perf4j.TimingLogger - start[1590649436812] time[0] tag[writeTile]
2020-05-28 09:03:56,813 [main] INFO  org.perf4j.TimingLogger - st

2020-05-28 09:03:56,866 [main] INFO  org.perf4j.TimingLogger - start[1590649436604] time[262] tag[convertToPyramid]
CPU times: user 28.5 ms, sys: 14.4 ms, total: 42.8 ms
Wall time: 1.23 s


In [9]:
!ls -ltrah /tmp/output*tiff

-rw-r--r--  1 jamoore  wheel    54K May 28 09:03 /tmp/output-1.ome.tiff
-rw-r--r--  1 jamoore  wheel   138M May 28 09:03 /tmp/output-2.ome.tiff


### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.